In [8]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [9]:
%cd \Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO

C:\Users\Sebas\OneDrive\Documenten\School\3e jaar\Project CO


In [10]:
"""
Enter filepath of instance and desired name for the solution file
"""
directory = "\\Users\\Sebas\\OneDrive\\Documenten\\School\\3e jaar\\Project CO"
filename = 'CO_Case2401.txt'


"""
 Read the instance and assign variables to the different values
"""

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)

Machines:
[[  1   7 315]
 [  2   6 578]]

Locations:
[[  1 230 367]
 [  2 349 370]
 [  3 297 246]
 [  4 495 150]
 [  5 376 373]]

Requests:
[[ 1  3  1  3  1  1]
 [ 2  2  1  4  2  2]
 [ 3  2  1  4  1  2]
 [ 4  4  1  3  1  2]
 [ 5  3  1  2  2  1]
 [ 6  2  1  3  2  2]
 [ 7  3  1  2  1  1]
 [ 8  3  1  3  1  2]
 [ 9  4  1  2  1  1]
 [10  2  1  3  1  2]]

Technicians:
[[  1   3 930   5   1   1]
 [  2   5 898   7   1   1]]
5
15
750
10
250
100000
5
400
20000


In [11]:
# main code 1, de distnace word niet goed geprint (overal 0)

def nearest_neighbor(point, unvisited_points, locations):
    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - distance: The Euclidean distance between the locations of the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)


def get_truck_routes_with_days(locations, depot, truck_capacity, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= truck_capacity:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= truck_capacity:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append(current_truck_route)
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            truck_routes.append(current_truck_route)
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append(current_truck_route)
    
    return truck_routes


def calculate_delivery_days(truck_routes, requests):
    delivery_days = {}
    for truck_route in truck_routes:
        for location_id, request_ids in enumerate(truck_route, start=1):
            for request_id in request_ids:
                delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
                delivery_days[request_id] = delivery_day  # Record the delivery day for each request
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes, requests)




## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)


def assign_technicians(delivery_dates, technicians, requests, max_consecutive_days):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_dates.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            location_id = technician[1]
            if technician[1] == location_id:
                skill_set = technician[4:]
                if skill_set[requests[request_id - 1][4] - 1] == 1:
                    earliest_available_day = max(earliest_assignment_day, installation_day)
                    
                    if technician[0] not in technician_assignments:
                        technician_assignments[technician[0]] = []
                        consecutive_days[technician[0]] = 0
                    
                    if consecutive_days[technician[0]] < max_consecutive_days:
                        if technician[0] not in consecutive_days:
                            consecutive_days[technician[0]] = 1
                        else:
                            consecutive_days[technician[0]] += 1
                            
                        technician_assignments[technician[0]].append((request_id, earliest_available_day))
                        break
                        
    return technician_assignments

# Example usage with the maximum consecutive days constraint
max_consecutive_days = 5

technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)



# Print technician assignments
for technician_id, assigned_requests in technician_assignments.items():
    technician = [tech for tech in technicians if tech[0] == technician_id][0]
    skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
    # Get the indices of the skill set where the value is 1
    skill_set_indices = [index + 1 for index, value in enumerate(skill_set) if value == 1]
    print(f"Technician {technician_id} skill set: {skill_set_indices} | Assigned Requests:")
    for request_id, assignment_day in assigned_requests:
        print(f"  - Request {request_id} (Earliest Assignment Day: {assignment_day})")





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass



def print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days, locations):
    total_distance_traveled = 0
    
    for idx, truck_route in enumerate(truck_routes, start=1):
        total_distance_truck = 0
        print(f"Truck {idx} route:")
        total_weight = 0
        
        for request_ids in truck_route:
            for i, request_id in enumerate(request_ids):
                if i > 0:
                    prev_request_id = request_ids[i - 1]
                    distance = calculate_distance_between_requests(prev_request_id, request_id, requests, locations)
                    if distance is not None:
                        total_distance_truck += distance
                        total_distance_traveled += distance  # Accumulate distance traveled across all trucks
                        print(f" (Distance: {distance:.2f})", end="")
                    else:
                        print(" (Distance calculation error)", end="")
                
                machine_kind_id = requests[request_id - 1][4]
                print(f"Request {request_id} (Idle days: {idle_days[machine_kind_id]}, Delivery Day: {delivery_days[request_id]})", end="")
                
                request_weight = calculate_request_weight(request_id, requests, machines)
                total_weight += request_weight
            
            print()
        
        print(f"  Total weight: {total_weight}")
        print(f"  Total distance traveled by Truck {idx}: {total_distance_truck:.2f} units\n")
    
    print(f"Total distance traveled by all trucks: {total_distance_traveled:.2f} units")



# Example usage
print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days, locations)


technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)
penalties = calculate_penalties(delivery_days, requests)
apply_constraints(technician_assignments)



# Example usage
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)


Location corresponding to Request ID 3: ID=2, Coordinates=(349, 370)
Location corresponding to Request ID 5: ID=3, Coordinates=(297, 246)
Distance between Request ID 3 and Request ID 5: 134.46
Requests belonging to Location ID 2: [2, 3, 6, 10]
Requests belonging to Location ID 3: [1, 5, 7, 8]
Requests belonging to Location ID 4: [4, 9]
Location ID 3: [1, 5, 7, 8]
Location ID 2: [2, 3, 6, 10]
Location ID 4: [4, 9]
Technician 1 skill set: [1, 2] | Assigned Requests:
  - Request 1 (Earliest Assignment Day: 2)
  - Request 2 (Earliest Assignment Day: 2)
  - Request 3 (Earliest Assignment Day: 2)
  - Request 4 (Earliest Assignment Day: 2)
  - Request 5 (Earliest Assignment Day: 2)
Technician 2 skill set: [1, 2] | Assigned Requests:
  - Request 6 (Earliest Assignment Day: 2)
  - Request 7 (Earliest Assignment Day: 2)
  - Request 8 (Earliest Assignment Day: 2)
  - Request 9 (Earliest Assignment Day: 2)
  - Request 10 (Earliest Assignment Day: 2)
Truck 1 route:
Request 1 (Idle days: 0, Delivery

In [12]:
# Print the schedule for trucks and technicians
def print_schedule(truck_routes, technician_assignments, delivery_days, requests):
    current_day = 1
    while True:
        print()  # Print an empty line before each day
        print(f"DAY = {current_day}")
        
        # Print truck routes for the current day
        truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
        num_trucks_for_day = len(truck_routes_for_day)
        if num_trucks_for_day > 0:
            print(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}")
            for truck_index, route in enumerate(truck_routes_for_day, start=1):
                request_ids = " ".join(str(request_id) for request_id in route)
                print(f"{truck_index} {request_ids}")
        else:
            print("NUMBER_OF_TRUCKS = 0")
        
        # Print technician assignments for the current day
        technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
        num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
        if num_technicians_for_day > 0:
            print(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}")
            for tech_id, request_id in technician_assignments_for_day:
                print(f"{tech_id} {request_id}")
        else:
            print("NUMBER_OF_TECHNICIANS = 0")
        
        # Check if there are no more routes or assignments for the next day
        if not truck_routes_for_day and not technician_assignments_for_day:
            break
        
        current_day += 1

# Example usage
print_schedule(truck_routes, technician_assignments, delivery_days, requests)


DAY = 1
NUMBER_OF_TRUCKS = 10
1 [1]
2 [2]
3 [3]
4 [4]
5 [5]
6 [6]
7 [7]
8 [8]
9 [9]
10 [10]
NUMBER_OF_TECHNICIANS = 0

DAY = 2
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 2
1 1
1 2
1 3
1 4
1 5
2 6
2 7
2 8
2 9
2 10

DAY = 3
NUMBER_OF_TRUCKS = 0
NUMBER_OF_TECHNICIANS = 0


In [13]:
def get_truck_routes_with_days(locations, depot, truck_capacity, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append(depot_loc_ID)  # Start at the depot
            current_truck_route.append(location_id)
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= truck_capacity:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            current_truck_route.append(location_id)
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= truck_capacity:
                current_truck_weight = combined_weight
            else:
                current_truck_route.append(depot_loc_ID)  # End at the depot
                truck_routes.append(current_truck_route)
                current_truck_route = [depot_loc_ID, location_id]  # Start a new route
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # end the current route at the depot and start a new route
            current_truck_route.append(depot_loc_ID)  # End at the depot
            truck_routes.append(current_truck_route)
            current_truck_route = [depot_loc_ID, location_id]  # Start a new route
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        current_truck_route.append(depot_loc_ID)  # End at the depot
        truck_routes.append(current_truck_route)
    
    return truck_routes

# Example usage
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)
print("Truck Routes:", truck_routes)


Truck Routes: [[1, 3, 1], [1, 2, 1], [1, 2, 1], [1, 4, 1], [1, 3, 1], [1, 2, 1], [1, 3, 1], [1, 3, 1], [1, 4, 1], [1, 2, 1]]


In [16]:
def calculate_truck_distance(truck_routes, locations):
    total_truck_distance = 0
    truck_distances = []
    distance_array = []  # Nieuwe array voor cumulatieve afstanden. Leek mij handig.
    for truck_route in truck_routes:
        truck_distance = 0
        for i in range(len(truck_route) - 1):
            location_id1 = truck_route[i]
            location_id2 = truck_route[i + 1]
            coordinates1 = locations[location_id1][1:]
            coordinates2 = locations[location_id2][1:]
            distance = calculate_distance(coordinates1, coordinates2)
            truck_distance += distance
            total_truck_distance += distance
            distance_array.append(total_truck_distance)
        truck_distances.append(truck_distance)
    return total_truck_distance, truck_distances, distance_array

# Generate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate truck distances
total_distance, truck_distances, distance_array = calculate_truck_distance(truck_routes, locations)

# Print each truck's distance
print("Distances traveled by trucks:")
for i, distance in enumerate(truck_distances, start=1):
    print(f"Truck {i}: {distance:.2f}")

# Print total distance
print(f"Total distance traveled by trucks: {total_distance:.2f}")

# Print cumulative distances, met stapsgewijs de toevoegingen
print("Cumulative distances:")
for i, distance in enumerate(distance_array, start=1):
    print(f"Step {i}: {distance:.2f}")


Distances traveled by trucks:
Truck 1: 528.08
Truck 2: 268.92
Truck 3: 268.92
Truck 4: 54.33
Truck 5: 528.08
Truck 6: 268.92
Truck 7: 528.08
Truck 8: 528.08
Truck 9: 54.33
Truck 10: 268.92
Total distance traveled by trucks: 3296.66
Cumulative distances:
Step 1: 264.04
Step 2: 528.08
Step 3: 662.54
Step 4: 797.00
Step 5: 931.46
Step 6: 1065.92
Step 7: 1093.09
Step 8: 1120.26
Step 9: 1384.29
Step 10: 1648.33
Step 11: 1782.79
Step 12: 1917.26
Step 13: 2181.29
Step 14: 2445.33
Step 15: 2709.37
Step 16: 2973.41
Step 17: 3000.57
Step 18: 3027.74
Step 19: 3162.20
Step 20: 3296.66


In [19]:
# main code 2: nieuwe berekening van distance, maar de rest wordt niet geprint omdat enumerate(request_ids) niet iterable is
# enige 2 veranderingen tov main 1: def get_truck_routes_with_days en def calculate_truck_distance


def nearest_neighbor(point, unvisited_points, locations):
    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_point in unvisited_points:
        candidate_point_id = int(candidate_point)
        if candidate_point_id != int(point):
            candidate_coordinates = get_location_coordinates(candidate_point_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = candidate_point_id
    return nearest_point

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - distance: The Euclidean distance between the locations of the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)


def get_truck_routes_with_days(locations, depot, truck_capacity, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append(depot_loc_ID)  # Start at the depot
            current_truck_route.append(location_id)
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= truck_capacity:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            current_truck_route.append(location_id)
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= truck_capacity:
                current_truck_weight = combined_weight
            else:
                current_truck_route.append(depot_loc_ID)  # End at the depot
                truck_routes.append(current_truck_route)
                current_truck_route = [depot_loc_ID, location_id]  # Start a new route
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # end the current route at the depot and start a new route
            current_truck_route.append(depot_loc_ID)  # End at the depot
            truck_routes.append(current_truck_route)
            current_truck_route = [depot_loc_ID, location_id]  # Start a new route
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        current_truck_route.append(depot_loc_ID)  # End at the depot
        truck_routes.append(current_truck_route)
    
    return truck_routes

truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)
print("Truck Routes:", truck_routes)

def calculate_truck_distance(truck_routes, locations):
    total_truck_distance = 0
    truck_distances = []
    distance_array = []  # Nieuwe array voor cumulatieve afstanden. Leek mij handig.
    for truck_route in truck_routes:
        truck_distance = 0
        for i in range(len(truck_route) - 1):
            location_id1 = truck_route[i]
            location_id2 = truck_route[i + 1]
            coordinates1 = locations[location_id1][1:]
            coordinates2 = locations[location_id2][1:]
            distance = calculate_distance(coordinates1, coordinates2)
            truck_distance += distance
            total_truck_distance += distance
            distance_array.append(total_truck_distance)
        truck_distances.append(truck_distance)
    return total_truck_distance, truck_distances #return hier even niet distance_array, anders moeten verwijzingen anders



def calculate_delivery_days(truck_routes, requests):
    delivery_days = {}
    for truck_route in truck_routes:
        for location_id, request_ids in enumerate(truck_route, start=1):
            for request_id in request_ids:
                delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
                delivery_days[request_id] = delivery_day  # Record the delivery day for each request
    return delivery_days


# Generate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate truck distances
total_distance, truck_distances = calculate_truck_distance(truck_routes, locations)

# Print each truck's distance
for i, distance in enumerate(truck_distances, start=1):
    print(f"Distance traveled by Truck {i}: {distance:.2f}")

# Print total distance
print(f"Total distance traveled by trucks: {total_distance:.2f}")




## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)


def assign_technicians(delivery_dates, technicians, requests, max_consecutive_days):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_dates.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            location_id = technician[1]
            if technician[1] == location_id:
                skill_set = technician[4:]
                if skill_set[requests[request_id - 1][4] - 1] == 1:
                    earliest_available_day = max(earliest_assignment_day, installation_day)
                    
                    if technician[0] not in technician_assignments:
                        technician_assignments[technician[0]] = []
                        consecutive_days[technician[0]] = 0
                    
                    if consecutive_days[technician[0]] < max_consecutive_days:
                        if technician[0] not in consecutive_days:
                            consecutive_days[technician[0]] = 1
                        else:
                            consecutive_days[technician[0]] += 1
                            
                        technician_assignments[technician[0]].append((request_id, earliest_available_day))
                        break
                        
    return technician_assignments

# Example usage with the maximum consecutive days constraint
max_consecutive_days = 5

technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)



# Print technician assignments
for technician_id, assigned_requests in technician_assignments.items():
    technician = [tech for tech in technicians if tech[0] == technician_id][0]
    skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
    # Get the indices of the skill set where the value is 1
    skill_set_indices = [index + 1 for index, value in enumerate(skill_set) if value == 1]
    print(f"Technician {technician_id} skill set: {skill_set_indices} | Assigned Requests:")
    for request_id, assignment_day in assigned_requests:
        print(f"  - Request {request_id} (Earliest Assignment Day: {assignment_day})")





# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass



def print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days, locations):
    total_distance_traveled = 0
    
    for idx, truck_route in enumerate(truck_routes, start=1):
        total_distance_truck = 0
        print(f"Truck {idx} route:")
        total_weight = 0
        
        for request_ids in truck_route:
            for i, request_id in enumerate(request_ids):
                if i > 0:
                    prev_request_id = request_ids[i - 1]
                    distance = calculate_distance_between_requests(prev_request_id, request_id, requests, locations)
                    if distance is not None:
                        total_distance_truck += distance
                        total_distance_traveled += distance  # Accumulate distance traveled across all trucks
                        print(f" (Distance: {distance:.2f})", end="")
                    else:
                        print(" (Distance calculation error)", end="")
                
                machine_kind_id = requests[request_id - 1][4]
                print(f"Request {request_id} (Idle days: {idle_days[machine_kind_id]}, Delivery Day: {delivery_days[request_id]})", end="")
                
                request_weight = calculate_request_weight(request_id, requests, machines)
                total_weight += request_weight
            
            print()
        
        print(f"  Total weight: {total_weight}")
        print(f"  Total distance traveled by Truck {idx}: {total_distance_truck:.2f} units\n")
    
    print(f"Total distance traveled by all trucks: {total_distance_traveled:.2f} units")



# Example usage
print_truck_routes(truck_routes, requests, machines, idle_days, delivery_days, locations)


technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days)
penalties = calculate_penalties(delivery_days, requests)
apply_constraints(technician_assignments)



# Example usage
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)


Location corresponding to Request ID 3: ID=2, Coordinates=(349, 370)
Location corresponding to Request ID 5: ID=3, Coordinates=(297, 246)
Distance between Request ID 3 and Request ID 5: 134.46
Requests belonging to Location ID 2: [2, 3, 6, 10]
Requests belonging to Location ID 3: [1, 5, 7, 8]
Requests belonging to Location ID 4: [4, 9]
Location ID 3: [1, 5, 7, 8]
Location ID 2: [2, 3, 6, 10]
Location ID 4: [4, 9]
Truck Routes: [[1, 3, 1], [1, 2, 1], [1, 2, 1], [1, 4, 1], [1, 3, 1], [1, 2, 1], [1, 3, 1], [1, 3, 1], [1, 4, 1], [1, 2, 1]]
Distance traveled by Truck 1: 528.08
Distance traveled by Truck 2: 268.92
Distance traveled by Truck 3: 268.92
Distance traveled by Truck 4: 54.33
Distance traveled by Truck 5: 528.08
Distance traveled by Truck 6: 268.92
Distance traveled by Truck 7: 528.08
Distance traveled by Truck 8: 528.08
Distance traveled by Truck 9: 54.33
Distance traveled by Truck 10: 268.92
Total distance traveled by trucks: 3296.66
Technician 1 skill set: [1, 2] | Assigned Requ

TypeError: 'numpy.int32' object is not iterable